In [203]:
import numpy as np 
import pandas as pd 
import sklearn 
import warnings
from sklearn import (preprocessing, model_selection)

In [146]:
# disable error yang muncul
def warn(*args, **kwargs):
    pass

warnings.warn = warn

In [147]:
data = pd.DataFrame({
    "kondisi_sil_master":["bocor", "normal", "normal", "bocor"],
    "kondisi_ring_piston":["kotor", "bersih", "bersih", "kotor"],
    "kondisi_kampas_rem":["tipis","normal", "normal", "tipis"],
    "nilai_voltase_aki":[3.5, 8.0, 4., 5.5],
    "umur_aki":[3,2,1,2],
    "kondisi_switch_starter":["mute", "normal", "mute", "normal"],
    "nilai_arang_starter":[0.2, 0.05, 0.15, 0.25],
    "kondisi_lahar_dinamo":["meleleh", "normal", "normal","meleleh"],
    "kondisi_dinamo":["hangus", "normal", "normal", "hangus"],
    "kondisi_bohlam":["putus", "normal", "normal", "putus"],
    "kondisi_timah_soket":["meleleh", "normal" ,"normal", "meleleh"],
    "kondisi_kabel_lampu":["putus", "normal", "normal", "putus"],
    "kondisi_saklar_lampu":["rusak", "normal", "normal", "rusak"],
    "kondisi_sepul":["rusak", "normal", "normal", "rusak"],
    "kondisi_regulator":["rusak", "normal", "normal", "rusak"],
    "kondisi_gas":["tidak_normal", "normal", "normal", "tidak_normal"],
    "kondisi_selongsong_gas":["berkarat", "normal", "normal", "berkarat"],
    "kondisi_tali_gas":["putus", "normal", "normal", "putus"],
    "usia_oli_saat_terakhir_ganti":[4, 8, 3, 5],
})

In [148]:
# lihat tipe data 
'''
REM ==> ganti rem
---
kondisi_sil_master
kondisi_ring_piston
kondisi_kampas_rem

AKI ganti aki
---
nilai_voltase_aki
umur_aki
kondisi_switch_starter

LAMPU ==> ganti lampu
----
kondisi_bohlam
kondisi_saklar_lampu
kondisi_kabel_lampu

GAS ==> Pengecekan lanjut
---
kondisi_gas
kondisi_selongsong_gas
kondisi_tali_gas
'''

# semua tipe data selain object, maka kita abaikan dulu
data.dtypes


kondisi_sil_master               object
kondisi_ring_piston              object
kondisi_kampas_rem               object
nilai_voltase_aki               float64
umur_aki                          int64
kondisi_switch_starter           object
nilai_arang_starter             float64
kondisi_lahar_dinamo             object
kondisi_dinamo                   object
kondisi_bohlam                   object
kondisi_timah_soket              object
kondisi_kabel_lampu              object
kondisi_saklar_lampu             object
kondisi_sepul                    object
kondisi_regulator                object
kondisi_gas                      object
kondisi_selongsong_gas           object
kondisi_tali_gas                 object
usia_oli_saat_terakhir_ganti      int64
dtype: object

In [149]:
def generate_string_randomly(data,column_name, n_record):
    '''
    fungsi untuk menghasilkan /membuat 
    record data baru berdasarkan jumlah unique nilai
    dalam suatu feature. ini dikhususkan untuk 
    feature dengan jenis object/string. 
    ------------------------------------------------
    return : array dataFrame
    '''
    
    data_prep = data[column_name].values
    # encode data yang string ke int/real
    enc = preprocessing.LabelEncoder()
    out_encoded = enc.fit_transform(data_prep)
    # mencari tahu berapa jenis nilai (unique) dalam suatu feature
    unique = np.unique(out_encoded)
    # hitung panjang unique
    length_unique = len(unique)
    np.random.seed(42) # supaya random nilai hanya sekali ketika fungsi dijalankan berkali2 
    # methode untuk generate nilai random 
    arr = np.random.randint(0, length_unique, size=n_record)
    # buat dataframe agar lebih mudah dieksekusi
    nama_field = str(column_name)
    nilai_field = arr
    dataFrame = pd.DataFrame({nama_field:nilai_field})
    # ganti semua data yang nilainya 0 dan 1 menjadi bocor dan normal secara berurutan
    out_df = dataFrame.replace(0, 'bocor', inplace=False)
    output = out_df.replace(1, 'normal', inplace=False)
    # ubah ke dalam mode vektor/list
    output = output.values.flatten()    
    # kembalikan nilai output
    return output 

In [150]:
def generate_number_randomly(data, column_name, low, high, size):
    '''
    fungsi untuk membuat data baru yang bertipe 
    float atau int secara automatis. fungsi ini
    telah automatis mendeteksi apakah tipe data
    yang dimasukan berjenis int atau float
    -------------------------------------------
    return dataFrame
    '''
    
    # ambil satu buah nilai data suatu kolom
    np.random.seed(42)
    out = data[column_name].to_list()[0]
    # kita gunakan try dan except untuk pencegahan error
#     try:
    '''
    jika tipe datanya adalah bertipe float maka lanjut,
    jika tidak, maka program akan mengesekusi blok 'except' 
    '''
    if type(out) == float:
        '''
        generate random nilai bertipe float dari low sampai high 
        dengan ukuran size
        '''
        float_gen = np.random.uniform(low, high, size=size)
        # bulatkan nilai float_gen menjadi dua angka di belakang koma
        float_gen = np.round(float_gen, decimals=2)
        # pastikan nama kolom berjenis string
        float_column = str(column_name)
        # masukan nilai float_gen hasil random tadi ke dalam dataFrame
        float_out = pd.DataFrame({float_column:float_gen})
        # konversi dataFrame ke dalam format array list
        float_out = float_out.values.flatten()
        # kembalikan nilai float_out
        return float_out
    else:
        '''
        generate random nilai bertipe int dari low sampai high 
        dengan ukuran size
        '''
        int_gen = np.random.randint(low=int(low), high=int(high), size=size)
        # pastikan nama kolom berjenis string
        int_column = str(column_name)
        # masukan nilai int_gen hasil random tadi ke dalam dataFrame
        int_out = pd.DataFrame({int_column:int_gen})
        # konversi dataFrame ke dalam format array list
        int_out = int_out.values.flatten()
        # kembalikan nilai int_out
        return int_out

In [151]:
# sipkan variable kosong untuk menampung semua nilai2 akhir hasil generation
output_storage = {} 

# panggil semua nama kolomnya secara cepat lalu masukan ke dalam 
# variable arr list_nama_kolom
list_nama_kolom = data.columns

# looping list_nama_kolom
# artinya kita mengambil nama kolomnya satu persatu untuk diparsing ke dalam 
# program yang ada di bawahnya
for col in list_nama_kolom:
    # kita deteksi tipe data dari suatu kolom/field/feature
    type_detector = type(data[col].to_list()[0])
    
    # jika datanya termasuk bertipe string, maka
    # panggil function generate_record_randomly
    if type_detector == str:
        
        # panggil fungsi untuk generate string/object
        output = generate_string_randomly(
                data=data,
                column_name=col, 
                n_record=100
        )
        output_storage.update({col:output})
        
    else:
        # panggil fungsi untuk generate number
        output = generate_number_randomly(
                    data=data,
                    column_name=col, 
                    low=1, 
                    high=10, 
                    size=100
                )
        output_storage.update({col:output})

In [152]:
# ini dia dataset barunya
data_akhir = pd.DataFrame(output_storage)

# jalankan kolom ini dengan menampilkan 10 baris pertama dari data
data_akhir.head(10)

,kondisi_sil_master,kondisi_ring_piston,kondisi_kampas_rem,nilai_voltase_aki,umur_aki,kondisi_switch_starter,nilai_arang_starter,kondisi_lahar_dinamo,kondisi_dinamo,kondisi_bohlam,kondisi_timah_soket,kondisi_kabel_lampu,kondisi_saklar_lampu,kondisi_sepul,kondisi_regulator,kondisi_gas,kondisi_selongsong_gas,kondisi_tali_gas,usia_oli_saat_terakhir_ganti
0,bocor,bocor,bocor,4.37,7,bocor,4.37,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,7
1,normal,normal,normal,9.56,4,normal,9.56,normal,normal,normal,normal,normal,normal,normal,normal,normal,normal,normal,4
2,bocor,bocor,bocor,7.59,8,bocor,7.59,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,8
3,bocor,bocor,bocor,6.39,5,bocor,6.39,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,5
4,bocor,bocor,bocor,2.40,7,bocor,2.40,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,7
5,normal,normal,normal,2.40,3,normal,2.40,normal,normal,normal,normal,normal,normal,normal,normal,normal,normal,normal,3
6,bocor,bocor,bocor,1.52,7,bocor,1.52,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,7
7,bocor,bocor,bocor,8.80,8,bocor,8.80,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,8
8,bocor,bocor,bocor,6.41,5,bocor,6.41,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,bocor,5
9,normal,normal,normal,7.37,4,normal,7.37,normal,normal,normal,normal,normal,normal,normal,normal,normal,normal,normal,4


## FEATURE SELECTION

In [153]:
'''
pilih 3 feature yang merepresetasikan satu objek class keputusan.
misalkan untuk ganti lampu, maka mekanik harus mendapatkan data
seperti kondisi_bohlam, kondisi_saklar dan kondisi_lampu. Hanya
3 feature/kolom saja. 
'''
data_selected = data_akhir[
                    [
                        'kondisi_sil_master', 'kondisi_ring_piston', 'kondisi_kampas_rem',
                        'nilai_voltase_aki', 'umur_aki', 'kondisi_switch_starter', 
                        'kondisi_bohlam', 'kondisi_saklar_lampu', 'kondisi_kabel_lampu',
                        'kondisi_gas', 'kondisi_selongsong_gas', 'kondisi_tali_gas'
                    ]
                ]  

In [154]:
# tampilkan 5 baris pertama dari datanya
data_selected.head(5)

,kondisi_sil_master,kondisi_ring_piston,kondisi_kampas_rem,nilai_voltase_aki,umur_aki,kondisi_switch_starter,kondisi_bohlam,kondisi_saklar_lampu,kondisi_kabel_lampu,kondisi_gas,kondisi_selongsong_gas,kondisi_tali_gas
0,bocor,bocor,bocor,4.37,7,bocor,bocor,bocor,bocor,bocor,bocor,bocor
1,normal,normal,normal,9.56,4,normal,normal,normal,normal,normal,normal,normal
2,bocor,bocor,bocor,7.59,8,bocor,bocor,bocor,bocor,bocor,bocor,bocor
3,bocor,bocor,bocor,6.39,5,bocor,bocor,bocor,bocor,bocor,bocor,bocor
4,bocor,bocor,bocor,2.40,7,bocor,bocor,bocor,bocor,bocor,bocor,bocor


In [155]:
target = ['ganti_rem', 'ganti_aki', 'ganti_lampu', 'cek_gas_lanjut']
data_keputusan = pd.DataFrame({'keputusan':target})
data_keputusan

,keputusan
0,ganti_rem
1,ganti_aki
2,ganti_lampu
3,cek_gas_lanjut


In [182]:
def generate_class_randomly(data, n_record):
    data_target = data['keputusan'].values
    # encode data yang string ke int/real
    enc = preprocessing.LabelEncoder()
    out_encoded = enc.fit_transform(data_target)
    # mencari tahu berapa jenis nilai (unique) dalam suatu feature
    unique = np.unique(out_encoded)
    # hitung panjang unique
    length_unique = len(unique)
    np.random.seed(42) # supaya random nilai hanya sekali ketika fungsi dijalankan berkali2 
    # methode untuk generate nilai random 
    arr = np.random.randint(0, length_unique, size=n_record)
    # buat dataframe agar lebih mudah dieksekusi
    column_name='keputusan'
    nama_field = str(column_name)
    nilai_field = arr
    dataFrame = pd.DataFrame({nama_field:nilai_field})
    '''
    ganti semua data yang nilainya 0, 1 ,2 dan 3 menjadi 
    ganti_rem, ganti_aki, ganti_lampu dan cek_gas_lanjut
    '''
    output = dataFrame.replace(0, 'ganti_rem', inplace=False)
    output = output.replace(1, 'ganti_aki', inplace=False)
    output = output.replace(2, 'ganti_lampu', inplace=False)
    output = output.replace(3, 'cek_gas_lanjut', inplace=False)

    # ubah ke dalam mode vektor/list
    output_target = output.values.flatten()    
    return output

In [183]:
# masukan kolom keputusan ke dalam data_selected 
data_selected['keputusan'] = generate_class_randomly(data=data_keputusan, n_record=100)
# dataset finalnya adalah 
dataset = data_selected

In [185]:
# dataset finalnya adalah 
dataset.head()

,kondisi_sil_master,kondisi_ring_piston,kondisi_kampas_rem,nilai_voltase_aki,umur_aki,kondisi_switch_starter,kondisi_bohlam,kondisi_saklar_lampu,kondisi_kabel_lampu,kondisi_gas,kondisi_selongsong_gas,kondisi_tali_gas,keputusan
0,bocor,bocor,bocor,4.37,7,bocor,bocor,bocor,bocor,bocor,bocor,bocor,ganti_lampu
1,normal,normal,normal,9.56,4,normal,normal,normal,normal,normal,normal,normal,cek_gas_lanjut
2,bocor,bocor,bocor,7.59,8,bocor,bocor,bocor,bocor,bocor,bocor,bocor,ganti_rem
3,bocor,bocor,bocor,6.39,5,bocor,bocor,bocor,bocor,bocor,bocor,bocor,ganti_lampu
4,bocor,bocor,bocor,2.40,7,bocor,bocor,bocor,bocor,bocor,bocor,bocor,ganti_lampu


## Transformasi

In [186]:
'''
kita pertama merubah bentuk data yang string di atas (normal, bocor dll)
menjadi number. karena komputer hanya bisa membaca number/biner. 
sehingga perlu untuk dikonversi terlebih dahulu. 
Untuk data-data yang sudah berbentuk number (float/int/real), maka tidak 
perlu dilakukan
'''

# siapkan datafram kosong
dataset_trans = pd.DataFrame({' ' : [] })

# inisialisasi object untuk proses transformasi (encoding)
transformasi = preprocessing.LabelEncoder()

# ambil semua nama kolomnya saja
columns = dataset.columns

# lalu looping satu satu perkolom supaya cepet dalam melakukan transformasi
for col in columns:
    
    # jika kolom yang terdeteksi memiliki tipe data str maka lakukan transformasi
    if type(dataset[col].tolist()[0]) == str:
        output = transformasi.fit_transform(dataset[col].values)
        dataset_trans[col] = output
    # jika tidak, maka abaikan
    else:
        dataset_trans[col] = dataset[col]
        
# lihat dataset hasil transformasi        
dataset_trans.head()

,,kondisi_sil_master,kondisi_ring_piston,kondisi_kampas_rem,nilai_voltase_aki,umur_aki,kondisi_switch_starter,kondisi_bohlam,kondisi_saklar_lampu,kondisi_kabel_lampu,kondisi_gas,kondisi_selongsong_gas,kondisi_tali_gas,keputusan
0,NaN,0,0,0,4.37,7,0,0,0,0,0,0,0,2
1,NaN,1,1,1,9.56,4,1,1,1,1,1,1,1,0
2,NaN,0,0,0,7.59,8,0,0,0,0,0,0,0,3
3,NaN,0,0,0,6.39,5,0,0,0,0,0,0,0,2
4,NaN,0,0,0,2.40,7,0,0,0,0,0,0,0,2


In [187]:
'''
jika kita lihat, terdapat nilai NaN pada kolom pertama
maka kita harus menghilangkannya
'''
dataset_trans = dataset_trans.iloc[:, 1:]
dataset_trans.head()

,kondisi_sil_master,kondisi_ring_piston,kondisi_kampas_rem,nilai_voltase_aki,umur_aki,kondisi_switch_starter,kondisi_bohlam,kondisi_saklar_lampu,kondisi_kabel_lampu,kondisi_gas,kondisi_selongsong_gas,kondisi_tali_gas,keputusan
0,0,0,0,4.37,7,0,0,0,0,0,0,0,2
1,1,1,1,9.56,4,1,1,1,1,1,1,1,0
2,0,0,0,7.59,8,0,0,0,0,0,0,0,3
3,0,0,0,6.39,5,0,0,0,0,0,0,0,2
4,0,0,0,2.40,7,0,0,0,0,0,0,0,2


## Data Splitting 

In [199]:
# import librarynya
from sklearn import model_selection
# pisahkan data dari kolom 'keputusan'
FEATURES = dataset_trans.iloc[:, :-1]
TARGET = dataset_trans.iloc[:, -1] # TARGET = KEPUTUSAN

'''
pecah dataset menjadi dua yaitu training dan testing dengan rasio pemcahan 80 : 20,
80 % untuk training dan 20 % untuk testing
'''
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    FEATURES, 
    TARGET, 
    test_size=0.2
)

In [202]:
# lihat ukuran/dimensi dari datanya setelah splitting data
print("Dimensi data (n_baris, n_kolom) \n")
print("Dimensi data training untuk feature-nya aja (x_train) : \n", x_train.shape)
print("Dimensi data training untuk target-nya  aja (y_train) : \n", y_train.shape)
print("Dimensi data testing  untuk feature-nya aja (x_test)  : \n", x_test.shape)
print("Dimensi data testing  untuk target-nya  aja (y_test)  : \n", y_test.shape)

Dimensi data (n_baris, n_kolom) 

Dimensi data training untuk feature-nya aja (x_train) : 
 (80, 12)
Dimensi data training untuk target-nya  aja (y_train) : 
 (80,)
Dimensi data testing  untuk feature-nya aja (x_test)  : 
 (20, 12)
Dimensi data testing  untuk target-nya  aja (y_test)  : 
 (20,)


## Memulai Training data

In [219]:
# import library algoritma pelatihannya
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

# import library untuk mengukur akurasi dan laporan hasil klasifikasi
from sklearn.metrics import accuracy_score, classification_report

# inisialisasi model-nya
models = {
    "Support_vector_machine": SVC(kernel="linear", gamma="auto"),
    "Decission_Tree": DecisionTreeClassifier(criterion='entropy'),
    "multi_layer_perceptron": MLPClassifier(hidden_layer_sizes=(250),max_iter=1000,batch_size=100)
}

# membuat fungsi untuk melakukan fungsi fitting atau training
def train_model (model,xtrain,ytrain):
    model.fit(xtrain,ytrain)

# membuat fungsi untuk menampilkan akurasi dan classifiation_report
def predict(i,model,xtest,ytest):
    pred = model.predict(xtest)
    accuracy = accuracy_score(ytest, pred) * 100
    print("\n",i,":")
    print(classification_report(ytest, pred, target_names=enc.classes_))
    print("accuracy : \n", round(accuracy,3),"%\n")

# melakukan training dengan menggunakan semua algoritma yang telah diinisialisasi
for i in models:
    train_model(models[i],x_train,y_train)
    predict(i,models[i],x_test,y_test)


 Support_vector_machine :
                precision    recall  f1-score   support

cek_gas_lanjut       0.62      0.62      0.62         8
     ganti_aki       0.25      0.25      0.25         4
   ganti_lampu       0.50      1.00      0.67         4
     ganti_rem       0.00      0.00      0.00         4

     micro avg       0.50      0.50      0.50        20
     macro avg       0.34      0.47      0.39        20
  weighted avg       0.40      0.50      0.43        20

accuracy : 
 50.0 %


 Decission_Tree :
                precision    recall  f1-score   support

cek_gas_lanjut       0.71      0.62      0.67         8
     ganti_aki       0.40      0.50      0.44         4
   ganti_lampu       0.60      0.75      0.67         4
     ganti_rem       0.67      0.50      0.57         4

     micro avg       0.60      0.60      0.60        20
     macro avg       0.60      0.59      0.59        20
  weighted avg       0.62      0.60      0.60        20

accuracy : 
 60.0 %


 multi_la

In [218]:
'''
kita melihat hasil prediksi dengan diberikan 10 data feature tanpa keputusan. 
nanti, outputnya berupa hasil prediksi sebanyak 10 baris setelah mengerapkan 
3 algrotima pelatihan tersebut
'''
data_test = x_test[:10]
target_test = y_test[:10]
model = models['multi_layer_perceptron']
model_svm = models['Support_vector_machine']
model_dec_tree = models['Decission_Tree']
pred_mlp = model.predict(data_test)
pred_svm = model_svm.predict(data_test)
pred_dec_tree = model_dec_tree.predict(data_test)
data_comp = pd.DataFrame({
    "Y_MLP":pred_mlp, 
    "Y_SVM":pred_svm, 
    "Y_DEC_TREE":pred_dec_tree, 
    "Y_TRUE":target_test
})
print("hasil prediksi setelah menggunakan ketiga algoritma")
data_comp.head(10)

hasil prediksi setelah menggunakan ketiga algoritma


,Y_MLP,Y_SVM,Y_DEC_TREE,Y_TRUE
98,0,0,0,1
91,1,1,1,0
36,0,0,0,0
44,1,0,1,0
67,3,2,3,3
21,2,2,2,3
19,3,2,2,2
29,0,0,1,0
82,2,2,3,3
5,0,0,0,0


<strong> Keterangan : </strong> <br>
0 = ganti_rem <br>
1 = ganti_aki <br>
2 = ganti_lampu <br>
3 = cek_gas_lanjut <br>

## Kesimpulan 
Dari 3 algoritma di atas, akurasi tertinggi didapatkan setelah menerapkan multi layer perceptron (neural network) dengan akurasi sebesar 65%

## Persiapan Production ke web service